In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sp
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse.linalg import spsolve

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
from sklearn.model_selection import train_test_split

In [2]:
events = pd.read_csv('C:/Users/kgoyal/Desktop/Bkp_new/COEWork/AI Hackathon/Retail Rocket/events.csv/events.csv')


In [3]:
events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [4]:
len(events.timestamp)

2756101

In [5]:
pd.crosstab(index=events["event"],columns="count")
len(events.itemid.unique())
len(events)

2756101

In [6]:
from pandasql import sqldf, load_meat, load_births

In [7]:
new_events = sqldf('Select visitorid,event,itemid, count(*) as times from events group by visitorid,event,itemid')

In [9]:
new_events.head()
len(new_events)

2215422

In [10]:
new_events['VisitorItem'] = new_events['visitorid'].astype(str) + ',' + new_events['itemid'].astype(str)
new_events.head()

,visitorid,event,itemid,times,VisitorItem
0,0,view,67045,1,"0,67045"
1,0,view,285930,1,"0,285930"
2,0,view,357564,1,"0,357564"
3,1,view,72028,1,"1,72028"
4,2,view,216305,2,"2,216305"


In [11]:
new_events2 = new_events.pivot( index ='VisitorItem' , columns='event', values='times')
new_events2.head()

event,addtocart,transaction,view
VisitorItem,,,
"0,285930",NaN,NaN,1.0
"0,357564",NaN,NaN,1.0
"0,67045",NaN,NaN,1.0
"1,72028",NaN,NaN,1.0
"10,248766",NaN,NaN,1.0


In [25]:
new_events3 = sqldf('Select * from new_events2')
new_events3.head()

,VisitorItem,addtocart,transaction,view
0,"0,285930",NaN,NaN,1.0
1,"0,357564",NaN,NaN,1.0
2,"0,67045",NaN,NaN,1.0
3,"1,72028",NaN,NaN,1.0
4,"10,248766",NaN,NaN,1.0


In [26]:
new_events3['visitorid'], new_events3['itemid'] = new_events3['VisitorItem'].str.split(',',1).str
new_events3.head()

,VisitorItem,addtocart,transaction,view,visitorid,itemid
0,"0,285930",NaN,NaN,1.0,0,285930
1,"0,357564",NaN,NaN,1.0,0,357564
2,"0,67045",NaN,NaN,1.0,0,67045
3,"1,72028",NaN,NaN,1.0,1,72028
4,"10,248766",NaN,NaN,1.0,10,248766


In [27]:
new_events3['addtocartb'] = np.where(new_events3['view']>=1, 1,0)

In [30]:
split = len(new_events3)
round(split*.6)

1287107

In [32]:
x_train = new_events3[0:round(split*.6)]

In [35]:
len(x_train)
x_train.head()

,VisitorItem,addtocart,transaction,view,visitorid,itemid,addtocartb
0,"0,285930",NaN,NaN,1.0,0,285930,1
1,"0,357564",NaN,NaN,1.0,0,357564,1
2,"0,67045",NaN,NaN,1.0,0,67045,1
3,"1,72028",NaN,NaN,1.0,1,72028,1
4,"10,248766",NaN,NaN,1.0,10,248766,1


In [36]:
pd.crosstab(index=x_train["addtocartb"],columns="count")

col_0,count
addtocartb,
0,7758
1,1279349


In [45]:
from keras.models import Sequential
model = Sequential()

In [57]:
from keras.layers import Dense

model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

In [63]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [64]:
model.fit(x_train.values, new_events3['addtocartb'].values, epochs=5,batch_size=32)

Epoch 1/5
2145179/2145179 [==============================] - 140s 65us/step - loss: nan - acc: 0.0000e+00
Epoch 2/5
2145179/2145179 [==============================] - 100s 46us/step - loss: nan - acc: 0.0000e+00
Epoch 3/5
2145179/2145179 [==============================] - 90s 42us/step - loss: nan - acc: 0.0000e+00
Epoch 4/5
2145179/2145179 [==============================] - 84s 39us/step - loss: nan - acc: 0.0000e+00
Epoch 5/5
2145179/2145179 [==============================] - 84s 39us/step - loss: nan - acc: 0.0000e+00


In [ ]:
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)

In [ ]:
classes = model.predict(x_test, batch_size=128)

In [17]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True))

NameError: name 'keras' is not defined